In [ ]:
import csv
import tqdm

import pandas as pd
import numpy as np

GT ANOMALY

In [ ]:

anomaly=[]
for i in range(1,11):
    ANOMALY=np.load(f'./../../OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/y_test{i}.npy',allow_pickle=True)
    for x in range(len(ANOMALY)):
        an=[]
        for v in range(len(ANOMALY[x])):
            an.append(ANOMALY[x][v])
        c=0
        a=[]
        while c<len(an):
            a.append(an[c])
            c=c+1

        an=np.array(a)

        anomaly.append(an)  
        
anomaly=np.array(anomaly)

SCORE

In [ ]:
SAMPLING_FACTOR=1
list_of_df=[]
for i in range(34):
    read=[pd.read_csv(f'{i}.csv', 
                          sep=';', 
                          parse_dates=True)]
    read=np.array(read).reshape(-1)
    c=0
    a=[]
    succ=SAMPLING_FACTOR
    prec=0
    value= [0 for x in range(len(read)*SAMPLING_FACTOR)]
    while c<len(read):
        for v in range(prec,succ):
            value[v]=read[c]
        c=c+1
        succ=succ+SAMPLING_FACTOR

        prec=prec+SAMPLING_FACTOR
    #downsampled
    
    
    list_of_df.append(value)

In [ ]:



header=['th_factor','method','value','F1','precision','recall','TP','TN','FP','FN']

with open(f'ELM_MI.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.25,0.5,0.4,0.1,0.05,0.6,0.7]
    for t in thresholding_factor:
        
        test_=[pd.read_csv(f'threshold_score.csv', 
                          sep=',', 
                          parse_dates=True)]

        score=np.array(test_).reshape(-1)

        #IQR
        q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
        iqr = q3 - q1
        IQR = q3 + t* iqr

        #MAD
        median = np.median(score)
        mad = 1.4826 * np.median(np.abs(score - median))
        MAD = median + t * mad

        #STD
        mean, std = np.mean(score), np.std(score)
        STD = mean + t * std

        method=[IQR,MAD,STD]
        
        for g in range(len(method)):
            TP=0
            TN=0
            FN=0
            FP=0

            for i in range(34):

                outlier=list_of_df[i]>method[g]
                
                prediction_ = (outlier.astype(int)==1).reshape(-1)
                true_= anomaly[i][:len(prediction_)]>=1

                TP = TP+(true_ & prediction_).sum()   
                TN = TN+(~true_ & ~prediction_).sum()  
                FP = FP+(~true_ & prediction_).sum()    
                FN = FN+(true_ & ~prediction_).sum()    

            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
            m=['IQR','MAD','STD']
            row=[t,m[g],method[g],f1,PREC,REC,TP,TN,FP,FN]
            writer.writerow(row)